In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex, hexcolor_str

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 4: Giant Squid ---</h2><p>You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you <em>can</em> see, however, is a giant squid that has attached itself to the outside of your submarine.</p>
<p>Maybe it wants to play <a href="https://en.wikipedia.org/wiki/Bingo_(American_version)" target="_blank">bingo</a>?</p>
<p>Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is <em>marked</em> on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board <em>wins</em>. (Diagonals don't count.)</p>
<p>The submarine has a <em>bingo subsystem</em> to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:</p>
<pre><code>7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11 0
8 2 23 4 24
21 9 14 16 7
6 10 3 18 5
1 12 20 15 19

3 15 0 2 22
9 18 13 17 5
19 8 7 25 23
20 11 10 24 4
14 21 16 12 6

14 21 17 24 4
10 16 15 9 19
18 8 23 26 20
22 11 13 6 5
2 0 12 3 7
</code></pre>

<p>After the first five numbers are drawn (<code>7</code>, <code>4</code>, <code>9</code>, <code>5</code>, and <code>11</code>), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):</p>
<pre><code>22 13 17 <em>11</em>  0         3 15  0  2 22        14 21 17 24  <em>4</em>
 8  2 23  <em>4</em> 24         <em>9</em> 18 13 17  <em>5</em>        10 16 15  <em>9</em> 19
21  <em>9</em> 14 16  <em>7</em>        19  8  <em>7</em> 25 23        18  8 23 26 20
 6 10  3 18  <em>5</em>        20 <em>11</em> 10 24  <em>4</em>        22 <em>11</em> 13  6  <em>5</em>
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  <em>7</em>
</code></pre>
<p>After the next six numbers are drawn (<code>17</code>, <code>23</code>, <code>2</code>, <code>0</code>, <code>14</code>, and <code>21</code>), there are still no winners:</p>
<pre><code>22 13 <em>17</em> <em>11</em>  <em>0</em>         3 15  <em>0</em>  <em>2</em> 22        <em>14</em> <em>21</em> <em>17</em> 24  <em>4</em>
 8  <em>2</em> <em>23</em>  <em>4</em> 24         <em>9</em> 18 13 <em>17</em>  <em>5</em>        10 16 15  <em>9</em> 19
<em>21</em>  <em>9</em> <em>14</em> 16  <em>7</em>        19  8  <em>7</em> 25 <em>23</em>        18  8 <em>23</em> 26 20
 6 10  3 18  <em>5</em>        20 <em>11</em> 10 24  <em>4</em>        22 <em>11</em> 13  6  <em>5</em>
 1 12 20 15 19        <em>14</em> <em>21</em> 16 12  6         <em>2</em>  <em>0</em> 12  3  <em>7</em>
</code></pre>
<p>Finally, <code>24</code> is drawn:</p>
<pre><code>22 13 <em>17</em> <em>11</em>  <em>0</em>         3 15  <em>0</em>  <em>2</em> 22        <em>14</em> <em>21</em> <em>17</em> <em>24</em>  <em>4</em>
 8  <em>2</em> <em>23</em>  <em>4</em> <em>24</em>         <em>9</em> 18 13 <em>17</em>  <em>5</em>        10 16 15  <em>9</em> 19
<em>21</em>  <em>9</em> <em>14</em> 16  <em>7</em>        19  8  <em>7</em> 25 <em>23</em>        18  8 <em>23</em> 26 20
 6 10  3 18  <em>5</em>        20 <em>11</em> 10 <em>24</em>  <em>4</em>        22 <em>11</em> 13  6  <em>5</em>
 1 12 20 15 19        <em>14</em> <em>21</em> 16 12  6         <em>2</em>  <em>0</em> 12  3  <em>7</em>
</code></pre>
<p>At this point, the third board <em>wins</em> because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: <code><em>14 21 17 24  4</em></code>).</p>
<p>The <em>score</em> of the winning board can now be calculated. Start by finding the <em>sum of all unmarked numbers</em> on that board; in this case, the sum is <code>188</code>. Then, multiply that sum by <em>the number that was just called</em> when the board won, <code>24</code>, to get the final score, <code>188 * 24 = <em>4512</em></code>.</p>
<p>To guarantee victory against the giant squid, figure out which board will win first. <em>What will your final score be if you choose that board?</em></p>
</article>


In [2]:
from more_itertools import one


example = """
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""


class BingoCard:
    def __init__(self, nr: int, s: str) -> None:
        self.nr = nr
        self.card = [
            [int(f) for f in re.split(r"\s+", r.strip())] for r in s.splitlines()
        ]

        self.marked = [[False] * 5 for _ in range(5)]
        self.cache = {self.card[r][c]: (r, c) for r, c in product(range(5), repeat=2)}

    def check_drawn(self, drawn: int) -> bool:
        if drawn not in self.cache:
            return False
        r, c = self.cache[drawn]
        self.marked[r][c] = True
        return all(self.marked[r][i] for i in range(5)) or all(
            self.marked[i][c] for i in range(5)
        )

    def score(self) -> int:
        return sum(
            self.card[r][c]
            for r, c in product(range(5), repeat=2)
            if not self.marked[r][c]
        )

    def __repr__(self) -> str:
        return f"Card: {self.nr} \n" + "\n".join(
            " ".join(
                hexcolor_str(
                    "#07e312" if self.marked[r][c] else "#e3c607",
                    f"{self.card[r][c]:3}",
                )
                for c in range(5)
            )
            for r in range(5)
        )


class Bingo:
    def __init__(self, s: str) -> None:
        draw, *cards = re.split(r"\n\s*\n", s.strip())
        self.draw = [int(i) for i in reversed(draw.split(","))]
        self.cards = [BingoCard(i, c) for i, c in enumerate(cards, start=1)]

    def play(self, do_print=None) -> int:
        drawn_queue = []
        print_goal = do_print.pop(0) if do_print is not None else -1

        while self.draw:
            drawn = self.draw.pop()
            winners = self.check_drawn(drawn)
            if do_print is not None:
                drawn_queue.append(drawn)
                if len(drawn_queue) == print_goal:
                    print()
                    print(
                        f"After {print_goal} drawn ({','.join(map(str,drawn_queue)) })"
                    )
                    print()
                    print(self)
                    print_goal = do_print.pop(0) if do_print else -1
                    drawn_queue.clear()
                if winners:
                    print(
                        f"\nThe winner is card {hexcolor_str('@FFD700', str(one(winners)))}, with a score {hexcolor_str('@FFD700', str(self.cards[one(winners)].score() * drawn))}."
                    )

            if winners:
                return self.cards[one(winners)].score() * drawn

        return -1

    def last_board_to_win(self, do_print=None) -> int:
        drawn_queue = []
        print_goal = do_print.pop(0) if do_print is not None else -1

        while self.draw:
            drawn = self.draw.pop()
            winners = self.check_drawn(drawn)
            if do_print is not None:
                drawn_queue.append(drawn)
                if len(drawn_queue) == print_goal:
                    print()
                    print(
                        f"After {print_goal} drawn ({','.join(map(str,drawn_queue)) })"
                    )
                    print()
                    print(self)
                    drawn_queue.clear()
                if winners:
                    print()
                    print(
                        f"After {len(drawn_queue)} drawn ({','.join(map(str,drawn_queue)) })"
                    )
                    for winner in winners:
                        print(f"Winner card {hexcolor_str('@FFD700', str(winner))}.")

            if winners:
                if len(self.cards) == 1:
                    return drawn * one(self.cards).score()

                self.cards = [c for i, c in enumerate(self.cards) if i not in winners]

        return -1

    def check_drawn(self, drawn: int) -> set[BingoCard]:
        return {i for i, card in enumerate(self.cards) if card.check_drawn(drawn)}

    def __repr__(self) -> str:
        return "\n\n".join(str(c) for c in self.cards)


def play_bingo(s: str, do_print=None) -> int:
    bingo = Bingo(s)
    return bingo.play(do_print)


assert play_bingo(example, [5, 6, 1]) == 4512


After 5 drawn (7,4,9,5,11)

Card: 1 
 22  13  17  11   0
  8   2  23   4  24
 21   9  14  16   7
  6  10   3  18   5
  1  12  20  15  19

Card: 2 
  3  15   0   2  22
  9  18  13  17   5
 19   8   7  25  23
 20  11  10  24   4
 14  21  16  12   6

Card: 3 
 14  21  17  24   4
 10  16  15   9  19
 18   8  23  26  20
 22  11  13   6   5
  2   0  12   3   7

After 6 drawn (17,23,2,0,14,21)

Card: 1 
 22  13  17  11   0
  8   2  23   4  24
 21   9  14  16   7
  6  10   3  18   5
  1  12  20  15  19

Card: 2 
  3  15   0   2  22
  9  18  13  17   5
 19   8   7  25  23
 20  11  10  24   4
 14  21  16  12   6

Card: 3 
 14  21  17  24   4
 10  16  15   9  19
 18   8  23  26  20
 22  11  13   6   5
  2   0  12   3   7

After 1 drawn (24)

Card: 1 
 22  13  17  11   0
  8   2  23   4  24
 21   9  14  16   7
  6  10   3  18   5
  1  12  20  15  19

Card: 2 
  3  15   0   2  22
  9  18  13  17   5
 19   8   7  25  23
 20  11  10  24   4
 14  21  16  12   6

Card: 3 
 14  21  17  24   4
 10  16  

In [3]:
with open("../input/day4.txt") as f:
    puzzle = f.read()


print(f"Part I: {play_bingo(puzzle)}")

Part I: 6592


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>6592</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>On the other hand, it might be wise to try a different strategy: <span title="That's 'cuz a submarine don't pull things' antennas out of their sockets when they lose. Giant squid are known to do that.">let the giant squid win</span>.</p>
<p>You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to <em>figure out which board will win last</em> and choose that one. That way, no matter which boards it picks, it will win for sure.</p>
<p>In the above example, the second board is the last to win, which happens after <code>13</code> is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to <code>148</code> for a final score of <code>148 * 13 = <em>1924</em></code>.</p>
<p>Figure out which board will win last. <em>Once it wins, what would its final score be?</em></p>
</article>

</main>


In [4]:
def play_last_board_to_win(s: str, do_print=None) -> int:
    bingo = Bingo(s)
    return bingo.last_board_to_win(do_print)


assert play_last_board_to_win(example) == 1924

In [5]:
print(f"Part II: { play_last_board_to_win(puzzle)}")

Part II: 31755


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>31755</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
